In [1]:
import json
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

seed = 783
np.random.seed(seed=seed)

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
from keras import regularizers
from keras import initializers

from keras.optimizers import RMSprop as rm
from keras.optimizers import Adagrad as ada
from keras.optimizers import SGD as sgd

with open('../data/general/matches_2017_2018_v1.json') as matches_json:
    matches = pd.read_json(matches_json)

data_height = 6000
data_width = 87
label_team_width = 3
label_player_width = 29

timestep = 5

mean = 0
std = 0
min_values = 0
max_values = 0

# min-max scaling
def min_max_scaling(data):
    global min_values
    global max_values
    min_values = np.amin(data, axis=(0,1), keepdims=True)
    max_values = np.amax(data, axis=(0,1), keepdims=True)
    return (data-min_values)/(max_values-min_values)

# normalizing
def normalize(data):
    global mean
    global std
    mean = np.mean(data, axis=(0,1), keepdims=True)
    std = np.std(data, axis=(0,1), keepdims=True)
    return (data - mean)/std

/home/ismail/anaconda2/envs/project/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Match (60746) there is no pass data record so, it extracted from the system

Match (116001326) there is n referee record so, it extracted from the system

----- Small data preparing BEGIN -----

Small data arrays loaded from previously saved numpy array

In [8]:
first_small_match_id = 60561
small_match_count = 12

x_small = np.ndarray(shape=(small_match_count, data_height, data_width), dtype=np.float)
y_team_small = np.ndarray(shape=(small_match_count, data_height, label_team_width), dtype=np.int)
y_player_small = np.ndarray(shape=(small_match_count, data_height, label_player_width), dtype=np.int)
x_pass_small = None
y_pass_small = None

for id in range(small_match_count):
    x_small[id] = np.load('../data/match_' + str(id + first_small_match_id) + '/x_data.npy')
    y_team_small[id] = np.load('../data/match_' + str(id + first_small_match_id) + '/y_team_data.npy')
    y_player_small[id] = np.load('../data/match_' + str(id + first_small_match_id) + '/y_player_data.npy')
    
    y_pass = np.load('../data/match_' + str(id + first_small_match_id) + '/y_pass_data.npy')
    y_time = np.load('../data/match_' + str(id + first_small_match_id) + '/y_pass_time.npy')
    x_pass = np.ndarray(shape=(y_pass.shape[0], timestep, data_width + label_team_width + label_player_width))
    
    for i, t in enumerate(y_time):
        x_pass[i] = np.concatenate((x_small[id][t-timestep:t], y_team_small[id][t-timestep:t], \
                                    y_player_small[id][t-timestep:t]), axis=1)
    if x_pass_small is None and y_pass_small is None:
        x_pass_small = x_pass
        y_pass_small = y_pass
    else:
        x_pass_small = np.concatenate((x_pass_small, x_pass), axis=0)
        y_pass_small = np.concatenate((y_pass_small, y_pass), axis=0)

Small data - 5 seconds timesteps arrays creation

In [9]:
x_small_ = np.ndarray(shape=(small_match_count*5400, timestep, data_width), dtype=np.float)
y_team_small_ = np.ndarray(shape=(small_match_count*5400, label_team_width), dtype=np.int)
y_player_small_ = np.ndarray(shape=(small_match_count*5400, label_player_width), dtype=np.int)

for i in range(len(x_small)):
    for j in range(5400):
        for k in range(timestep):
            x_small_[i*5400 + j][k] = x_small[i][60 + j + k - timestep-1]
        y_team_small_[i*5400 + j] = y_team_small[i][j]
        y_player_small_[i*5400 + j] = y_player_small[i][j]
x_small_ = normalize(x_small_)

Small data - Subtracting the useless data, final match duration is [1,91]

In [10]:
x_small = x_small[:, 60:5460, :]
y_team_small = y_team_small[:, 60:5460, :]
y_player_small = y_player_small[:, 60:5460, :]

Small data - Normalize the data

In [11]:
x_small = normalize(x_small)

with open('../model/mean_std.pkl', 'wb') as file:
    pickle.dump((mean, std), file, pickle.HIGHEST_PROTOCOL)

x_pass_small = normalize(x_pass_small)

with open('../model/mean_std_pass.pkl', 'wb') as file:
    pickle.dump((mean, std), file, pickle.HIGHEST_PROTOCOL)

Small data - Fit the data [0,1]

In [7]:
x_small = min_max_scaling(x_small)

with open('../model/min_max.pkl', 'wb') as file:
    pickle.dump((min_values, max_values), file, pickle.HIGHEST_PROTOCOL)
    
x_pass_small = min_max_scaling(x_pass_small)

with open('../model/min_max_pass.pkl', 'wb') as file:
    pickle.dump((min_values, max_values), file, pickle.HIGHEST_PROTOCOL)

Small data - Shuffled the data

In [12]:
indices = np.arange(len(x_small))
np.random.shuffle(indices)

x_small = x_small[indices]
y_team_small = y_team_small[indices]
y_player_small = y_player_small[indices]

indices_pass = np.arange(len(x_pass_small))
np.random.shuffle(indices_pass)

x_pass_small = x_pass_small[indices_pass]
y_pass_small = y_pass_small[indices_pass]

Small data - Split the data train(%80) test(%20)

In [13]:
size_80 = int(x_small.shape[0]*0.8)

x_train_small = x_small[:size_80]
x_test_small = x_small[size_80:]

y_train_team_small = y_team_small[:size_80]
y_test_team_small = y_team_small[size_80:]

y_train_player_small = y_player_small[:size_80]
y_test_player_small = y_player_small[size_80:]

size_80 = int(x_pass_small.shape[0]*0.8)

x_train_pass_small = x_pass_small[:size_80]
x_test_pass_small = x_pass_small[size_80:]

y_train_pass_small = y_pass_small[:size_80]
y_test_pass_small = y_pass_small[size_80:]

----- Small data preparing END -----

----- All data preparing BEGIN -----

All data arrays (304 match - 243 of them train, 61 of them test)

In [2]:
x_all = np.ndarray(shape=(len(matches['id']), data_height, data_width))
y_team_all = np.ndarray(shape=(len(matches['id']), data_height, label_team_width))
y_player_all = np.ndarray(shape=(len(matches['id']), data_height, label_player_width))
x_pass_all = None
y_pass_all = None

idx = 0
for id in matches['id']:
    x_all[idx] = np.load('../data/match_' + str(id) + '/x_data.npy')
    y_team_all[idx] = np.load('../data/match_' + str(id) + '/y_team_data.npy')
    y_player_all[idx] = np.load('../data/match_' + str(id) + '/y_player_data.npy')
    
    y_pass = np.load('../data/match_' + str(id) + '/y_pass_data.npy')
    y_time = np.load('../data/match_' + str(id) + '/y_pass_time.npy')
    x_pass = np.ndarray(shape=(y_pass.shape[0], timestep, data_width + label_team_width + label_player_width))
    
    for i, t in enumerate(y_time):
        x_pass[i] = np.concatenate((x_all[idx][t-timestep:t], y_team_all[idx][t-timestep:t], \
                                    y_player_all[idx][t-timestep:t]), axis=1)
    if x_pass_all is None and y_pass_all is None:
        x_pass_all = x_pass
        y_pass_all = y_pass
    else:
        x_pass_all = np.concatenate((x_pass_all, x_pass), axis=0)
        y_pass_all = np.concatenate((y_pass_all, y_pass), axis=0)
    idx +=1

All data - Preparing the 5 seconds timesteps data

In [3]:
x_all_ = np.ndarray(shape=(len(matches['id'])*5400, timestep, data_width), dtype=np.float)
y_team_all_ = np.ndarray(shape=(len(matches['id'])*5400, label_team_width), dtype=np.int)
y_player_all_ = np.ndarray(shape=(len(matches['id'])*5400, label_player_width), dtype=np.int)

for i in range(len(matches['id'])):
    for j in range(5400):
        for k in range(timestep):
            x_all_[i*5400 + j][k] = x_all[i][60 + j + k - timestep-1]
        y_team_all_[i*5400 + j] = y_team_all[i][j]
        y_player_all_[i*5400 + j] = y_player_all[i][j]
#x_all_ = normalize(x_all_)

All data - Subtracting useless data final match duration is [1,91]

In [4]:
x_all = x_all[:, 60:5460, :]
y_team_all = y_team_all[:, 60:5460, :]
y_player_all = y_player_all[:, 60:5460, :]

All data - Normalize the input arrays

In [5]:
x_all = normalize(x_all)

with open('../model/mean_std.pkl', 'wb') as file:
    pickle.dump((mean, std), file, pickle.HIGHEST_PROTOCOL)
    
x_pass_all = normalize(x_pass_all)

with open('../model/mean_std_pass.pkl', 'wb') as file:
    pickle.dump((mean, std), file, pickle.HIGHEST_PROTOCOL)

All data - Fit the input arrays in [0,1]

In [ ]:
x_all = min_max_scaling(x_all)

with open('../model/min_max.pkl', 'wb') as file:
    pickle.dump((min_values, max_values), file, pickle.HIGHEST_PROTOCOL)

x_pass_all = min_max_scaling(x_pass_all)

with open('../model/min_max_pass.pkl', 'wb') as file:
    pickle.dump((min_values, max_values), file, pickle.HIGHEST_PROTOCOL)

All data - Shuffled the data

In [6]:
indices = np.arange(len(x_all))
np.random.shuffle(indices)

x_all = x_all[indices]
y_team_sall = y_team_all[indices]
y_player_all = y_player_all[indices]

indices_pass = np.arange(len(x_pass_all))
np.random.shuffle(indices_pass)

x_pass_all = x_pass_all[indices_pass]
y_pass_all = y_pass_all[indices_pass]

All data - Split the data train(%80) test(%20)

In [7]:
size_80 = int(x_all.shape[0]*0.8)

x_train_all = x_all[:size_80]
x_test_all = x_all[size_80:]

y_train_team_all = y_team_all[:size_80]
y_test_team_all = y_team_all[size_80:]

y_train_player_all = y_player_all[:size_80]
y_test_player_all = y_player_all[size_80:]

size_80 = int(x_pass_all.shape[0]*0.8)

x_train_pass_all = x_pass_all[:size_80]
x_test_pass_all = x_pass_all[size_80:]

y_train_pass_all = y_pass_all[:size_80]
y_test_pass_all = y_pass_all[size_80:]

----- All data preparing END -----

----- Team Ball Possesion BEGIN -----

Team Ball Possesion - Feed Forward trail

In [9]:
#x = x_train_small
x = x_train_all

#y = y_train_team_small
y = y_train_team_all

timestep = 5400

batch_size = 100
epochs = 30

input_dim = 87
output_dim = 3

model_team = Sequential()
model_team.add(Dense(1000, 
                     input_shape=(timestep, input_dim), 
                     kernel_initializer=initializers.he_normal(seed=seed), 
                     kernel_regularizer=regularizers.l2(0.01),
                     activation='relu'))
model_team.add(BatchNormalization())
model_team.add(Dense(500, 
                     kernel_initializer=initializers.he_normal(seed=seed), 
                     kernel_regularizer=regularizers.l2(0.01),
                     activation='relu'))
model_team.add(BatchNormalization())
model_team.add(Dense(250, 
                     kernel_initializer=initializers.he_normal(seed=seed), 
                     kernel_regularizer=regularizers.l2(0.01),
                     activation='relu'))
model_team.add(BatchNormalization())
model_team.add(Dense(100, 
                     kernel_initializer=initializers.he_normal(seed=seed), 
                     kernel_regularizer=regularizers.l2(0.01),
                     activation='relu'))
model_team.add(BatchNormalization())
model_team.add(Dense(50, 
                     kernel_initializer=initializers.he_normal(seed=seed), 
                     kernel_regularizer=regularizers.l2(0.01),
                     activation='relu'))
model_team.add(BatchNormalization())
model_team.add(Dense(output_dim, 
                     kernel_initializer=initializers.he_normal(seed=seed), 
                     kernel_regularizer=regularizers.l2(0.01),
                     activation='softmax'))

#opt = rm()
#opt = ada()
opt = sgd(lr=1e-3, momentum=0.9, decay=1e-5, nesterov=True)

model_team.compile(loss='categorical_crossentropy',
                   optimizer=opt,
                   metrics=['accuracy'])

history_team = model_team.fit(x=x, y=y, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=None, 
                              validation_split=0.2, validation_data=None, shuffle=True, class_weight=None, 
                              sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Train on 194 samples, validate on 49 samples
Epoch 1/30


Team Ball Possesion - Return sequence false trial

In [42]:
#x = x_small_
x = x_all_

#y = y_team_small_
y = y_team_all_

timestep = 5

batch_size = 100
epochs = 30

input_dim = 87
output_dim = 3

model_team = Sequential()
model_team.add(LSTM(6,
                    input_shape=(timestep, input_dim), 
                    return_sequences=False,
                    kernel_initializer=initializers.glorot_normal(seed=seed),
                    kernel_regularizer=regularizers.l2(0.01)))
model_team.add(BatchNormalization())
model_team.add(Dense(output_dim, 
                     kernel_initializer=initializers.he_normal(seed=seed), 
                     kernel_regularizer=regularizers.l2(0.01),
                     activation='softmax'))

#opt = rm()
#opt = ada()
opt = sgd(lr=1e-3, momentum=0.9, decay=1e-5, nesterov=True)

model_team.compile(loss='categorical_crossentropy',
                   optimizer=opt,
                   metrics=['accuracy'])

history_team = model_team.fit(x=x, y=y, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=None, 
                              validation_split=0.2, validation_data=None, shuffle=True, class_weight=None, 
                              sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Train on 51840 samples, validate on 12960 samples
Epoch 1/30


 - 9s - loss: 1.0363 - acc: 0.4849 - val_loss: 1.1787 - val_acc: 0.3770


Epoch 2/30


 - 6s - loss: 0.9163 - acc: 0.5826 - val_loss: 1.2895 - val_acc: 0.3589


Epoch 3/30


...

 - 6s - loss: 0.3855 - acc: 0.8517 - val_loss: 3.8605 - val_acc: 0.3075


Epoch 29/30


 - 6s - loss: 0.3786 - acc: 0.8546 - val_loss: 3.9637 - val_acc: 0.3136


Epoch 30/30


 - 6s - loss: 0.3705 - acc: 0.8586 - val_loss: 4.0940 - val_acc: 0.3081


Team Ball Possesion - Return sequence true trial

In [9]:
#x = x_train_small
x = x_train_all

#y = y_train_team_small
y = y_train_team_all

timestep = 5400

batch_size = 10
epochs = 100

input_dim = 87
output_dim = 3

model_team = Sequential()
model_team.add(LSTM(6, 
                    input_shape=(timestep, input_dim), 
                    return_sequences=True, 
                    kernel_initializer=initializers.glorot_normal(seed=seed),
                    kernel_regularizer=regularizers.l2(0.01)))
model_team.add(BatchNormalization())
model_team.add(Dense(output_dim, 
                     kernel_initializer=initializers.he_normal(seed=seed), 
                     kernel_regularizer=regularizers.l2(0.01),
                     activation='softmax'))

#opt = rm()
opt = ada()
#opt = sgd(lr=1e-2, momentum=0.9, decay=1e-10, nesterov=True)

model_team.compile(loss='categorical_crossentropy',
                   optimizer=opt,
                   metrics=['accuracy'])

history_team = model_team.fit(x=x, y=y, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=None, 
                              validation_split=0.2, validation_data=None, shuffle=True, class_weight=None, 
                              sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Train on 194 samples, validate on 49 samples
Epoch 1/100


 - 117s - loss: 1.4635 - acc: 0.3451 - val_loss: 1.3504 - val_acc: 0.3628


Epoch 2/100


 - 118s - loss: 1.2944 - acc: 0.3793 - val_loss: 1.2657 - val_acc: 0.3823


Epoch 3/100


...

Epoch 16/100


 - 117s - loss: 1.1136 - acc: 0.4424 - val_loss: 1.1210 - val_acc: 0.4332


Epoch 17/100


 - 119s - loss: 1.1120 - acc: 0.4422 - val_loss: 1.1191 - val_acc: 0.4342


Epoch 18/100


Team Ball Possesion - Evaluate the test set

In [ ]:
#x_test = x_test_small
x_test = x_test_all

#y_test = y_test_team_small
y_test = y_test_team_all

score = model_team.evaluate(x_test, y_test)
print('team possesion test acc: ', score[1])

3/3 [==============================] - 1s 402ms/step


team possesion test acc:  0.5325308442115784


Team Ball Possesion - Predict and save the test set results

In [ ]:
y_team_pred = model_team.predict(x_test)

Team Ball Possesion - Plot the learning curve

In [23]:
# summarize history for accuracy
plt.plot(history_team.history['acc'])
plt.plot(history_team.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.savefig('../graphs/team_acc.png')
plt.show()

# summarize history for loss
plt.plot(history_team.history['loss'])
plt.plot(history_team.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig('../graphs/team_loss.png')
plt.show()

Team Ball Possesion - Save the model and weights

In [15]:
model_team.save('../model/team_model.h5')
model_team.save_weights('../model/team_model_weight.h5')
json_data = model_team.to_json()

with open('../model/team_json_data.txt', 'w') as outfile:
    json.dump(json_data, outfile)

----- Team Ball Possesion END -----

----- Player Ball Possesion BEGIN -----

Player Ball Possesion - Return sequence false trial

In [13]:
#x = x_small_
x = x_all_

#y = y_player_small_
y = y_player_all_

timestep = 5

batch_size = 100
epochs = 30

input_dim = 87
output_dim = 29

model_player = Sequential()
model_player.add(LSTM(30,
                      input_shape=(timestep, input_dim), 
                      return_sequences=False, 
                      kernel_initializer=initializers.glorot_normal(seed=seed),
                      kernel_regularizer=regularizers.l2(0.01)))
model_player.add(BatchNormalization())
model_player.add(Dense(output_dim, 
                       kernel_initializer=initializers.he_normal(seed=seed), 
                       kernel_regularizer=regularizers.l2(0.01),
                       activation='softmax'))

#opt = rm()
#opt = ada()
opt = sgd(lr=1e-3, momentum=0.9, decay=1e-5, nesterov=True)

model_player.compile(loss='categorical_crossentropy',
                   optimizer=opt,
                   metrics=['accuracy'])

history_player = model_player.fit(x=x, y=y, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=None, 
                              validation_split=0.2, validation_data=None, shuffle=True, class_weight=None, 
                              sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Train on 51840 samples, validate on 12960 samples
Epoch 1/30


 - 7s - loss: 3.0354 - acc: 0.3010 - val_loss: 2.8775 - val_acc: 0.3788


Epoch 2/30


 - 5s - loss: 2.5802 - acc: 0.4462 - val_loss: 2.7996 - val_acc: 0.4202


Epoch 3/30


...

 - 5s - loss: 2.4453 - acc: 0.4482 - val_loss: 2.7755 - val_acc: 0.4248


Epoch 6/30


 - 5s - loss: 2.4239 - acc: 0.4489 - val_loss: 2.7824 - val_acc: 0.4238


Epoch 7/30


KeyboardInterrupt: 

Player Ball Possesion - Return sequence true trial

In [22]:
#x = np.concatenate((x_train_small, y_train_team_small), axis=2)
x = np.concatenate((x_train_all, y_train_team_all), axis=2)

#y = y_train_player_small
y = y_train_player_all

timestep = 5400

batch_size = 10
epochs = 30

input_dim = 90
output_dim = 29

model_player = Sequential()
model_player.add(LSTM(30, 
                      input_shape=(timestep, input_dim), 
                      return_sequences=True,
                      kernel_initializer=initializers.he_normal(seed=seed),
                      kernel_regularizer=regularizers.l2(0.01)))
model_player.add(BatchNormalization())
model_player.add(Dense(output_dim, kernel_initializer=initializers.he_normal(seed=seed), activation='softmax'))

#opt = rm()
#opt = ada()
opt = sgd(lr=5e-3, momentum=0.9, decay=1e-5, nesterov=True)

model_player.compile(loss='categorical_crossentropy',
                     optimizer=opt,
                     metrics=['accuracy'])

history_player = model_player.fit(x=x, y=y, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=None, 
                                  validation_split=0.2, validation_data=None, shuffle=True, class_weight=None, 
                                  sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Train on 7 samples, validate on 2 samples
Epoch 1/30


 - 101s - loss: 4.0903 - acc: 0.0284 - val_loss: 4.2662 - val_acc: 0.0294


Epoch 2/30


KeyboardInterrupt: 

Player Ball Possesion - Evaluate the test set

In [21]:
#x_test = np.concatenate((x_test_small, y_test_team_small), axis=2)
x_test = np.concatenate((x_test_all, y_test_team_all), axis=2)

#y_test = y_test_player_small
y_test = y_test_player_all

score = model_player.evaluate(x_test, y_test)
print('team possesion test acc: ', score[1])

32/61 [==============>...............] - ETA: 2s

61/61 [==============================] - 5s 77ms/step


team possesion test acc:  0.43678810264243456


Player Ball Possesion - Predict the test set results

In [ ]:
y_player_pred = model_player.predict(x_test)

Player Ball Possesion - Plot the learning curve

In [22]:
# summarize history for accuracy
plt.plot(history_player.history['acc'])
plt.plot(history_player.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.savefig('../graphs/player_acc.png')
plt.show()

# summarize history for loss
plt.plot(history_player.history['loss'])
plt.plot(history_player.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig('../graphs/player_loss.png')
plt.show()

Player Ball Possesion - Save the models and weights

In [23]:
model_player.save('../model/player_model.h5')
model_player.save_weights('../model/player_model_weight.h5')
json_data = model_player.to_json()

with open('../model/player_json_data.txt', 'w') as outfile:
    json.dump(json_data, outfile)

----- Player Ball Possesion END -----

----- Pass BEGIN -----

Pass - Return sequence false only this tried

In [24]:
#x = x_train_pass_small
x = x_train_pass_all

#y = y_train_pass_small
y = y_train_pass_all

batch_size = 100
epochs = 100
timestep = 5

input_dim = 119
output_dim = 28

model_pass = Sequential()
model_pass.add(LSTM(30, 
                    input_shape=(timestep, input_dim), 
                    return_sequences=False,
                    kernel_initializer=initializers.he_normal(seed=seed),
                    kernel_regularizer=regularizers.l2(0.01)))
model_pass.add(BatchNormalization())
model_pass.add(Dense(output_dim, 
                     kernel_initializer=initializers.he_normal(seed=seed), 
                     kernel_regularizer=regularizers.l2(0.01),
                     activation='softmax'))

#opt = rm()
#opt = ada()
opt = sgd(lr=1e-3, momentum=0.9, decay=1e-5, nesterov=True)

model_pass.compile(loss='categorical_crossentropy',
                   optimizer=opt,
                   metrics=['accuracy'])

history_pass = model_pass.fit(x=x, y=y, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=None, 
                              validation_split=0.2, validation_data=None, shuffle=True, class_weight=None, 
                              sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Train on 142829 samples, validate on 35708 samples
Epoch 1/100


 - 27s - loss: 2.6836 - acc: 0.1420 - val_loss: 2.4799 - val_acc: 0.1789


Epoch 2/100


 - 25s - loss: 2.3977 - acc: 0.2091 - val_loss: 2.4300 - val_acc: 0.1994


Epoch 3/100


...

 - 26s - loss: 1.1935 - acc: 0.6047 - val_loss: 4.0799 - val_acc: 0.1581


Epoch 99/100


 - 25s - loss: 1.1893 - acc: 0.6056 - val_loss: 4.1139 - val_acc: 0.1546


Epoch 100/100


 - 25s - loss: 1.1907 - acc: 0.6052 - val_loss: 4.1182 - val_acc: 0.1563


Pass - Evaluate the tests set

In [26]:
#x_test = x_test_pass_small
x_test = x_test_pass_all

#y_test = y_test_pass_small
y_test = y_test_pass_all

score = model_pass.evaluate(x_test, y_test)
print('next pass prediction test acc: ', score[1])

   32/44635 [..............................] - ETA: 7s

  352/44635 [..............................] - ETA: 7s

  768/44635 [..............................] - ETA: 6s

 1120/44635 [..............................] - ETA: 6s

 1440/44635 [..............................] - ETA: 6s

...

43552/44635 [============================>.] - ETA: 0s

43936/44635 [============================>.] - ETA: 0s

44288/44635 [============================>.] - ETA: 0s

44635/44635 [==============================] - 6s 143us/step


next pass prediction test acc:  0.15398230088562345


Pass - Predict the test set results

In [ ]:
y_pass_pred = model_pass.predict(x_test)

Pass - Plot the learning curve

In [27]:
# summarize history for accuracy
plt.plot(history_pass.history['acc'])
plt.plot(history_pass.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.savefig('../graphs/pass_acc.png')
plt.show()

# summarize history for loss
plt.plot(history_pass.history['loss'])
plt.plot(history_pass.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig('../graphs/pass_loss.png')
plt.show()

Pass - Save the model and weights

In [41]:
model_pass.save('../model/pass_model.h5')
model_pass.save_weights('../model/pass_model_weight.h5')
json_data = model_pass.to_json()

with open('../model/pass_json_data.txt', 'w') as outfile:
    json.dump(json_data, outfile)

----- Pass BEGIN -----